In [1]:
import sys
import os

import pandas as pd

sys.path.append(os.path.abspath('..'))
from src.env import PACKAGE_DIR
from configulator.config import default_config
from src.utils.call_api.schema import EmbeddingEngineEnum
from src.retrievers.models.bm25_retriever import KeywordRetriever
from src.retrievers.models.vector_retriever import VectorRetriever
from src.retrievers.models.ensemble_retriever import EnsembleRetriever
from src.evaluator.eval_retriever.evaluator import RetieverEvaluator

pd.set_option("display.max_colwidth", 500)
pd.set_option('display.max_columns', None)

In [2]:
cfg = default_config()
QUERY_FILE = PACKAGE_DIR/'data/raw/valid_sets/query_ans_txt.csv'
query_df = pd.read_csv(QUERY_FILE)
index_df = pd.read_csv(PACKAGE_DIR/'data/database/valid_sets/chartext_chunk200_lap0.2/openai_large.csv')
augindex_df = pd.read_csv(PACKAGE_DIR/'data/database/valid_sets/chartext_chunk200_lap20/openai_large_augmented.csv')

In [4]:
retrieve_target_df = augindex_df
keyretriever = KeywordRetriever(retrieve_target_df, tokenized_column_name='augmented_tokenized_text', target_column_name='augmented_text')
vecretriever = VectorRetriever(retrieve_target_df, emb_column_name='augmented_embedding', target_column_name='augmented_text', model=EmbeddingEngineEnum.Large)
ensretriever = EnsembleRetriever(vecretriever, keyretriever)

100%|██████████| 1017/1017 [00:00<00:00, 21001.57it/s]


In [6]:
def eval_retriever(retriever, top_k, query_df, return_miss: bool = False, evidence_column = 'similar_document',  **search_params):
    retrieved_texts_list = []
    for query in query_df['problem'].values:
        retrieved_texts = retriever.retrieve(query, top_k=top_k, **search_params)
        retrieved_texts_list.append(retrieved_texts[evidence_column])
    evaluator = RetieverEvaluator(query_df['evidence'], retrieved_texts_list)
    recall_score = evaluator.compute_mean_recall()

    if return_miss:
        return recall_score, evaluator.failed_retrieved_evidence

    return recall_score

In [22]:
eval_retriever(keyretriever, 5, query_df, return_miss=True, require_columns=['chunk_id', 'text'], evicence_column='text')

(0.4523809523809524,
 ['吉田は肺が悪い',
  '首縊りの縄',
  '鼠の仔を捕って来て入れてそれを黒焼きにしたもの',
  'あの荒物屋の娘が死んだと',
  '母親の話によるとそれは青物を売りに来る女があって、その女といろいろ話をしているうちにその肺病の特効薬の話をその女がはじめたというのだった',
  'その女には肺病の弟があってそれが死んでしまった',
  'いきなりその眼の前へ顔を近付けて、非常に押し殺した力強い声で、「心臓へ来ましたか？」と耳打ちをした女があった',
  '「その病気に利くええ薬を教えたげまひょか」',
  'ひょっとしてあなたは肺がお悪いのじゃありませんか',
  '吉田はそんな話を聞くにつけても、そういう迷信を信じる人間の無智に馬鹿馬鹿しさを感じないわけにいかなかったけれども、考えてみれば人間の無智というのはみな程度の差で、そう思って馬鹿馬鹿しさの感じを取り除いてしまえば、あとに残るのはそれらの人間の感じている肺病に対する手段の絶望と、病人達のなんとしてでも自分のよくなりつつあるという暗示を得たいという二つの事柄なのであった'])

In [23]:
eval_retriever(vecretriever, 5, query_df, return_miss=True, require_columns=['chunk_id', 'text'], evidence_column='text')

(0.40476190476190477,
 ['素焼の土瓶へ鼠の仔を捕って来て入れてそれを黒焼きにしたもの',
  '人間の脳味噌の黒焼き',
  '首縊りの縄',
  '鼠の仔を捕って来て入れてそれを黒焼きにしたもの',
  'あの荒物屋の娘が死んだと',
  '母親の話によるとそれは青物を売りに来る女があって、その女といろいろ話をしているうちにその肺病の特効薬の話をその女がはじめたというのだった',
  'その女には肺病の弟があってそれが死んでしまった',
  'それをすすめた人間は大和で塗師をしている男でその縄をどうして手に入れたかという話を吉田にして聞かせた',
  '「その病気に利くええ薬を教えたげまひょか」',
  'ひょっとしてあなたは肺がお悪いのじゃありませんか',
  '吉田はそんな話を聞くにつけても、そういう迷信を信じる人間の無智に馬鹿馬鹿しさを感じないわけにいかなかったけれども、考えてみれば人間の無智というのはみな程度の差で、そう思って馬鹿馬鹿しさの感じを取り除いてしまえば、あとに残るのはそれらの人間の感じている肺病に対する手段の絶望と、病人達のなんとしてでも自分のよくなりつつあるという暗示を得たいという二つの事柄なのであった'])

In [11]:
for vec in [0.1, 0.2, 0.3, 0.4, 0.5]:
    score = eval_retriever(ensretriever, 50, query_df, weights=[vec, 1-vec], ensemble_method='rrf', rank_impact_mitigator=60, require_columns=['chunk_id', 'text'], target_column='text')
    print(f'score is {score} when vec: {vec}, key: {1-vec}')

score is 0.8571428571428571 when vec: 0.1, key: 0.9
score is 0.8571428571428571 when vec: 0.2, key: 0.8
score is 0.8571428571428571 when vec: 0.3, key: 0.7
score is 0.8333333333333333 when vec: 0.4, key: 0.6
score is 0.8333333333333333 when vec: 0.5, key: 0.5


In [14]:
q_id = 1
print(query_df.loc[q_id, ['problem', 'evidence']])
keyretriever.retrieve(query_df.loc[q_id, 'problem'], top_k=10, require_columns=['chunk_id', 'text'])

problem     小説「のんきな患者」で、吉田が病院の食堂で出会った付添婦が勧めた薬の材料は何ですか？
evidence     「素焼《すやき》の土瓶《どびん》へ鼠の仔を捕って来て入れてそれを黒焼きにしたもの」
Name: 1, dtype: object


,similar_document,similarity,chunk_id,text
0,吉田が病院の食堂で何をしていたときに女に耳打ちされたのか？,12.744584,79,また吉田はその前の年母親が重い病気にかかって入院したとき一緒にその病院へついて行っていたことがあった。そのとき吉田がその病舎の食堂で、何心なく食事した後ぼんやりと窓に映る風景を眺めていると、いきなりその眼の前へ顔を近付けて、非常に押し殺した力強い声で、\n「心臓へ来ましたか？」\nと耳打ちをした女があった
1,その女が食堂に集まる他の付添婦たちを牛耳っていた理由は何だと考えられますか？,11.507057,80,はっとして吉田がその女の顔を見ると、それはその病舎の患者の付添いに雇われている付添婦の一人で、勿論そんな付添婦の顔触れにも毎日のように変化はあったが、その女はその頃露悪的な冗談を言っては食堂へ集まって来る他の付添婦たちを牛耳っていた中婆さんなのだった。\n吉田はそう言われて何のことかわからずにしばらく相手の顔を見ていたが、すぐに「ああなるほど」と気のついたことがあった
2,その薬の材料である鼠の仔はどのように捕まえるのか？,11.211693,84,するとその女はまたこんなことを言って吉田を閉口させてしまうのだった。\n「それは今ここで教えてもこの病院ではできまへんで」\nそしてそんな物々しい駄目をおしながらその女の話した薬というのは、素焼の土瓶へ鼠の仔を捕って来て入れてそれを黒焼きにしたもので、それをいくらか宛かごく少ない分量を飲んでいると、「一匹食わんうちに」癒るというのであった
3,その病舎の患者の付添いに雇われている付添婦たちはどのように変化しているのでしょうか？,10.755921,80,はっとして吉田がその女の顔を見ると、それはその病舎の患者の付添いに雇われている付添婦の一人で、勿論そんな付添婦の顔触れにも毎日のように変化はあったが、その女はその頃露悪的な冗談を言っては食堂へ集まって来る他の付添婦たちを牛耳っていた中婆さんなのだった。\n吉田はそう言われて何のことかわからずにしばらく相手の顔を見ていたが、すぐに「ああなるほど」と気のついたことがあった
4,吉田はどのようにして付添婦の背景を推測したのか？,10.465851,86,そして吉田が病院へ来て以来最もしみじみした印象をうけていたものはこの付添婦という寂しい女達の群れのことであって、それらの人達はみな単なる生活の必要というだけではなしに、夫に死に別れたとか年が寄って養い手がないとか、どこかにそうした人生の不幸を烙印されている人達であることを吉田は観察していたのであるが、あるいはこの女もそうした肉親をその病気で、なくすることによって、今こんなにして付添婦などをやっているのではあるまいかということを、吉田はそのときふと感じたのだった
5,吉田が付添婦たちを観察した理由は何か？,10.465851,86,そして吉田が病院へ来て以来最もしみじみした印象をうけていたものはこの付添婦という寂しい女達の群れのことであって、それらの人達はみな単なる生活の必要というだけではなしに、夫に死に別れたとか年が寄って養い手がないとか、どこかにそうした人生の不幸を烙印されている人達であることを吉田は観察していたのであるが、あるいはこの女もそうした肉親をその病気で、なくすることによって、今こんなにして付添婦などをやっているのではあるまいかということを、吉田はそのときふと感じたのだった
6,付添婦たちの共通点は何か？,9.703428,86,そして吉田が病院へ来て以来最もしみじみした印象をうけていたものはこの付添婦という寂しい女達の群れのことであって、それらの人達はみな単なる生活の必要というだけではなしに、夫に死に別れたとか年が寄って養い手がないとか、どこかにそうした人生の不幸を烙印されている人達であることを吉田は観察していたのであるが、あるいはこの女もそうした肉親をその病気で、なくすることによって、今こんなにして付添婦などをやっているのではあるまいかということを、吉田はそのときふと感じたのだった
7,付添婦たちの存在が吉田に与えた影響は何か？,9.667016,86,そして吉田が病院へ来て以来最もしみじみした印象をうけていたものはこの付添婦という寂しい女達の群れのことであって、それらの人達はみな単なる生活の必要というだけではなしに、夫に死に別れたとか年が寄って養い手がないとか、どこかにそうした人生の不幸を烙印されている人達であることを吉田は観察していたのであるが、あるいはこの女もそうした肉親をその病気で、なくすることによって、今こんなにして付添婦などをやっているのではあるまいかということを、吉田はそのときふと感じたのだった
8,薬をほとんど服用していない患者はどのくらいの割合ですか？,9.568037,106,つまりそれは、今非常に多くの肺結核患者が死に急ぎつつある。そしてそのなかで人間の望み得る最も行き届いた手当をうけている人間は百人に一人もないくらいで、そのうちの九十何人かはほとんど薬らしい薬ものまずに死に急いでいるということであった
9,その女はどのような役割を持っている付添婦ですか？,8.901584,80,はっとして吉田がその女の顔を見ると、それはその病舎の患者の付添いに雇われている付添婦の一人で、勿論そんな付添婦の顔触れにも毎日のように変化はあったが、その女はその頃露悪的な冗談を言っては食堂へ集まって来る他の付添婦たちを牛耳っていた中婆さんなのだった。\n吉田はそう言われて何のことかわからずにしばらく相手の顔を見ていたが、すぐに「ああなるほど」と気のついたことがあった


In [16]:
vecretriever.retrieve('主人公の吉田の患部は主にどこですか', top_k=10, require_columns=['chunk_id', 'text'])

,similar_document,similarity,chunk_id,text
0,吉田の身体が不安を感じ始めたのはなぜか？,0.636790,23,しかし母親を起こすことを考えると、こんな感情を抑えておそらく何度も呼ばなければならないだろうという気持だけでも吉田はまったく大儀な気になってしまうのだった。――しばらくして吉田はこの間から自分で起こしたことのなかった身体をじりじり起こしはじめた。そして床の上へやっと起きかえったかと思うと、寝床の上に丸くなって寝ている猫をむんずと掴まえた。吉田の身体はそれだけの運動でもう浪のように不安が揺れはじめた
1,吉田が苦痛を感じる理由は何か？,0.613134,13,吉田は何度「己が気持よく寝られさえすれば」と思ったことかしれなかった。こんな不安も吉田がその夜を睡むる当てさえあればなんの苦痛でもないので、苦しいのはただ自分が昼にも夜にも睡眠ということを勘定に入れることができないということだった。吉田は胸のなかがどうにかして和らんで来るまでは否でも応でもいつも身体を鯱硬張らして夜昼を押し通していなければならなかった
2,吉田はどのような病気を抱えているのか？,0.610218,1,吉田は肺が悪い。寒になって少し寒い日が来たと思ったら、すぐその翌日から高い熱を出してひどい咳になってしまった。胸の臓器を全部押し上げて出してしまおうとしているかのような咳をする。四五日経つともうすっかり痩せてしまった。咳もあまりしない。しかしこれは咳が癒ったのではなくて、咳をするための腹の筋肉がすっかり疲れ切ってしまったからで、彼らが咳をするのを肯じなくなってしまったかららしい
3,吉田の心身の状態は、どのような病気に関連している可能性があるのか？,0.601914,6,いったいひどく心臓でも弱って来たんだろうか、それともこんな病気にはあり勝ちな、不安ほどにはないなにかの現象なんだろうか、それとも自分の過敏になった神経がなにかの苦痛をそういうふうに感じさせるんだろうか。――吉田はほとんど身動きもできない姿勢で身体を鯱硬張らせたままかろうじて胸へ呼吸を送っていた。そして今もし突如この平衡を破るものが現われたら自分はどうなるかしれないということを思っていた
4,吉田が感じた苦痛の原因は何か？,0.599293,17,そしてこんな場合になっては吉田はやはり一匹の猫ぐらいでその母親を起こすということはできがたい気がするのだった。吉田はまた猫のことには「こんなことがあるかもしれないと思ってあんなにも神経質に言ってあるのに」と思って自分が神経質になることによって払った苦痛の犠牲が手応えもなくすっぽかされてしまったことに憤懣を感じないではいられなかった
5,吉田が身体を起こすことに対してどのような葛藤があったのか？,0.591343,23,しかし母親を起こすことを考えると、こんな感情を抑えておそらく何度も呼ばなければならないだろうという気持だけでも吉田はまったく大儀な気になってしまうのだった。――しばらくして吉田はこの間から自分で起こしたことのなかった身体をじりじり起こしはじめた。そして床の上へやっと起きかえったかと思うと、寝床の上に丸くなって寝ている猫をむんずと掴まえた。吉田の身体はそれだけの運動でもう浪のように不安が揺れはじめた
6,吉田はどのような状況で身体を硬張らせているのか？,0.589012,13,吉田は何度「己が気持よく寝られさえすれば」と思ったことかしれなかった。こんな不安も吉田がその夜を睡むる当てさえあればなんの苦痛でもないので、苦しいのはただ自分が昼にも夜にも睡眠ということを勘定に入れることができないということだった。吉田は胸のなかがどうにかして和らんで来るまでは否でも応でもいつも身体を鯱硬張らして夜昼を押し通していなければならなかった
7,吉田が抱える苦痛はどのようにして増していくのですか？,0.586533,8,その原因を臆測するにもまたその正否を判断するにも結局当の自分の不安の感じに由るほかはないのだとすると、結局それは何をやっているのかわけのわからないことになるのは当然のことなのだったが、しかしそんな状態にいる吉田にはそんな諦めがつくはずはなく、いくらでもそれは苦痛を増していくことになるのだった。\n第二に吉田を苦しめるのはこの不安には手段があると思うことだった
8,吉田が深い苦痛に陥る原因は何か？,0.583096,7,だから吉田の頭には地震とか火事とか一生に一度遭うか二度遭うかというようなものまでが真剣に写っているのだった。また吉田がこの状態を続けてゆくというのには絶えない努力感の緊張が必要であって、もしその綱渡りのような努力になにか不安の影が射せばたちどころに吉田は深い苦痛に陥らざるを得ないのだった。――しかしそんなことはいくら考えても決定的な知識のない吉田にはその解決がつくはずはなかった
9,吉田が身体に感じた不安はどのようなものであったのか？,0.579413,24,吉田の身体はそれだけの運動でもう浪のように不安が揺れはじめた。しかし吉田はもうどうすることもできないので、いきなりそれをそれの這入って来た部屋の隅へ「二度と手間のかからないように」叩きつけた。そして自分は寝床の上であぐらをかいてそのあとの恐ろしい呼吸困難に身を委せたのだった。\nしかし吉田のそんな苦しみもだんだん耐えがたいようなものではなくなって来た
